In [18]:
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.stats
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import torch.optim as optim

In [5]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
submisssion = pd.read_csv("./sample_submission.csv")

In [8]:
X = train.iloc[:, 1:]/255.
y = train.iloc[:, 0]
test = test/255.
# reshaping 784 to 28,28,1
X = X.values.reshape(train.shape[0], 28, 28, 1)
test = test.values.reshape(test.shape[0], 28, 28, 1)

In [11]:
# one hot of target values using keras's to_categorical class
y = to_categorical(y)

# splits train/test set
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, test_size=0.2, random_state=29)

# Shape
Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape

((33600, 28, 28, 1), (33600, 10), (8400, 28, 28, 1), (8400, 10))

In [13]:
# in model i am using BatchNormalization so for first step, i menualy normalize the batch
mean_px = Xtrain.mean().astype(np.float32)
std_px = Xtrain.std().astype(np.float32)

def standardize(x):
    return (x-mean_px)/std_px


In [16]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [20]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

network = Model()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)